# Data Loading

In [7]:
import pandas as pd
df = pd.read_csv("winemag-data-130k-v2.csv", index_col=0)

In [10]:
wine_descriptions = df['description'].tolist()

## Check how other models will be accurate with finding most simillar descriptions

# Sentence Transformers (SBERT) with models all-mpnet-base-v2, all-MiniLM-L6-v2

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

wine_descriptions = df['description'].tolist()

model = SentenceTransformer('all-mpnet-base-v2')
embeddings = model.encode(wine_descriptions)
embeddings = embeddings.astype(np.float32, copy=False)
np.save("embeddings_all-mpnet-base-v2.npy", embeddings)

c:\Users\Milosz\Desktop\studia\ml\projekt\projektUczenieMaszynowe\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


KeyboardInterrupt: 

In [7]:
from sentence_transformers import SentenceTransformer
import numpy as np

wine_descriptions = df['description'].tolist()

model = SentenceTransformer('all-mpnet-base-v2')
embeddings = model.encode(wine_descriptions)
embeddings = embeddings.astype(np.float32, copy=False)
np.save("embeddings_all-mpnet-base-v2.npy", embeddings)

c:\Users\Milosz\Desktop\studia\ml\projekt\projektUczenieMaszynowe\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Milosz\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' pa

# Open Ai API embeddings

Check how accurate will embeddings from LLM like Open Ai be

In [2]:
%pip install openai

  Using cached openai-2.15.0-py3-none-any.whl.metadata (29 kB)
  Using cached anyio-4.12.1-py3-none-any.whl.metadata (4.3 kB)
  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.12.0-cp313-cp313-win_amd64.whl.metadata (5.3 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached httpcore-1.0.9-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.16.0-py3-none-any.whl.metadata (8.3 kB)
Using cached openai-2.15.0-py3-none-any.whl (1.1 MB)
Using cached anyio-4.12.1-py3-none-any.whl (113 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.9-py3-none-any.whl (78 kB)
Using cached jiter-0.12.0-cp313-cp313-win_amd64.whl (204 kB)
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------------------------------------- 2.0/2.0 MB 22.1 MB/s eta 0:00:00
Using cached h11-0.16.0-py3-none-any.whl (37 kB)
Using cached sniffio-1.3.1-py3-none-any.whl (10 kB)

   -----------------------------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
import openai
import numpy as np
from tqdm import tqdm  # Pasek postępu
from private import API_KEY

client = openai.OpenAI(api_key=API_KEY)
MODEL = "text-embedding-3-small"
BATCH_SIZE = 2000

def get_embeddings_batched(texts, model=MODEL, batch_size=BATCH_SIZE):
    all_embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i : i + batch_size]
        
        try:
            batch = [text.replace("\n", " ") for text in batch]
            
            response = client.embeddings.create(input=batch, model=model)
            
            batch_embeddings = [data.embedding for data in response.data]
            all_embeddings.extend(batch_embeddings)
            
        except Exception as e:
            print(f"Błąd przy paczce {i}: {e}")
            break

    return all_embeddings

print(f"Embeddings from wine_descriptions {len(wine_descriptions)}")
embeddings_list = get_embeddings_batched(wine_descriptions)
embeddings = np.array(embeddings_list).astype(np.float32, copy=False)
np.save("embeddings_open_ai_api.npy", embeddings)


Embeddings from wine_descriptions 129971


In [12]:
np.save("embeddings_open_ai_api.npy", embeddings)